# Depreciation to FERC 1 Connection

This stage in the connection process is very much under constuction

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np
import pudl
import sqlalchemy as sa
import logging
import sys

from datetime import date

import pudl_rmi
from pudl_rmi.connect_deprish_to_ferc1 import *

import pudl_rmi.model_inputs

import warnings
warnings.filterwarnings('ignore')

In [ ]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)
handler = logging.StreamHandler(stream=sys.stdout)
formatter = logging.Formatter('%(message)s')
handler.setFormatter(formatter)
logger.handlers = [handler]

pd.options.display.max_columns = None
pd.options.display.max_rows = 60

## Make the output via rmi_out

In [ ]:
# pudl output object
pudl_settings = pudl.workspace.setup.get_defaults()
pudl_engine = sa.create_engine(pudl_settings["pudl_db"])
pudl_out = pudl.output.pudltabl.PudlTabl(
    pudl_engine,freq='AS',
    fill_fuel_cost=True,
    roll_fuel_cost=True,
    fill_net_gen=True,
)
rmi_out = pudl_rmi.coordinate.Output(
    pudl_out,
)

In [ ]:
ppl, d, de, fe, scaled_df = rmi_out.grab_all(clobber_all=False)
# temp-ish methods that I don't know where to put yet
# i haven't checked all of them in but i need them for RMI's current
# output
de = pudl_rmi.make_plant_parts_eia.reassign_id_ownership_dupes(de)
fe = pudl_rmi.connect_ferc1_to_eia.calc_annual_capital_addts_ferc1(fe)
try:
    de = pudl_rmi.model_inputs.fake_duke_deprish_eia_for_mod(de).pipe(pudl.helpers.convert_cols_dtypes, 'eia')
except AttributeError:
    logger.info("this is not checked in rn")

### Test

In [ ]:
df_test = pd.DataFrame(
    {
        'record_id_test': [
            'record_1', 'record_2', 'record_3',
        ],
        'plant_id_eia': [
            1, 2, 3,
        ],
        'report_date': [
            '2020-01-01', '2020-01-01', '2020-01-01',
        ],
        'plant_part': [
            'plant', 'plant_prime_mover', 'plant_gen',
        ],
        'operational_status_pudl': [
            'operating', 'operating', 'operating',
        ],
        'ferc_acct_name': [
            pd.NA, pd.NA, pd.NA,
        ],
        'unit_id_pudl': [
            pd.NA, pd.NA, pd.NA,
        ],
        'technology_description': [
            pd.NA, pd.NA, 'Conventional Steam Coal',
        ],
        'generator_id': [
            pd.NA, pd.NA, "a",
        ],
        'prime_mover_code': [
            pd.NA, 'ST', 'ST',
        ],
        'energy_source_code_1': [
            pd.NA, pd.NA, pd.NA,
        ],
        'data_col': [
            300, 100, 10,
        ],
        'utility_id_eia': [
            1, 1, 1,
        ],
        'ownership': [
            'total', 'total', 'total',
        ],
        'record_id_eia': [
            'record_id_1_1a_1b_1c', 'reocrd_id_2_2a_2b', 'record_id_3_a',
        ],
    }
).pipe(pudl.helpers.convert_cols_dtypes, 'eia').assign(report_year=lambda x: x.report_date.dt.year)

ppl_test = pd.DataFrame(
    {
        'plant_id_eia': [
            1, 1, 1,
            2, 2,
            3,
        ],
        'report_date': [
            '2020-01-01', '2020-01-01', '2020-01-01',
            '2020-01-01', '2020-01-01',
            '2020-01-01',
        ],
        'plant_part': [
            'plant_gen', 'plant_gen', 'plant_gen',
            'plant_gen', 'plant_gen',
            'plant_gen',
        ],
        'operational_status_pudl': [
            'operating', 'operating', 'operating',
            'operating', 'operating',
            'operating',
        ],
        'ferc_acct_name': [
            'steam', 'steam', 'steam',
            'steam', 'steam',
            'steam',
        ],
        'unit_id_pudl': [
            pd.NA, pd.NA, pd.NA,
            pd.NA, pd.NA,
            pd.NA,
        ],
        'technology_description': [
            'Conventional Steam Coal', 'Conventional Steam Coal',
            'Conventional Steam Coal',
            'Conventional Steam Coal', 'Conventional Steam Coal',
            'Conventional Steam Coal',
        ],
        'generator_id': [
            "1a", "1b", "1c",
            "2a", "2b",
            "a",
        ],
        'prime_mover_code': [
            'ST', 'ST', 'ST',
            'ST', 'ST',
            'ST',
        ],
        'energy_source_code_1': [
            'BIT', 'BIT', 'BIT',
            'BIT', 'BIT',
            'BIT',
        ],
        'utility_id_eia': [
            1, 1, 1,
            1, 1,
            1,
        ],
        'ownership': [
            'total', 'total', 'total',
            'total', 'total',
            'total',
        ],
        'record_id_eia': [
            'record_id_1_1a', 'record_id_1_1b', 'record_id_1_1c',
            'record_id_2_2a', 'record_id_2_2b',
            'record_id_3_a',
        ],
        'capacity_mw': [
            50, 30, 20,
            10, 10,
            100,
        ],
        'net_generation_mwh': [
            10000, 1000, 100,
            250, 200,
            5000,
        ],
        'total_fuel_cost': [
            500, 100, 50,
            250, 200,
            300,
        ],
    }
).pipe(pudl.helpers.convert_cols_dtypes, 'eia').assign(report_year=lambda x: x.report_date.dt.year)

In [ ]:
# scale_to_plant_part(
#     scale_part='plant_gen',
#     df_to_scale=df_test,
#     ppl=ppl_test,
#     data_set_idx_cols=['record_id_test'],
#     data_cols=['data_col'],
#     other_cols_to_keep=[],
# ).reset_index().to_dict()

# scale_part='plant_gen'
# df_to_scale=df_test.pipe(pudl.helpers.convert_cols_dtypes, 'eia')
# ppl=ppl_test.pipe(pudl.helpers.convert_cols_dtypes, 'eia')
# data_set_idx_cols=['record_id_test']
# data_cols=['data_col']
# other_cols_to_keep=[]

### Scale Everything to the Generator-level & Merge

In [ ]:
ferc_deprish_eia = pudl_rmi.connect_deprish_to_ferc1.execute(
    plant_parts_eia=ppl,
    deprish_eia=de,
    ferc1_to_eia=fe
)

#### Asset Retirement Cost stuff

In [ ]:
# There are some temp functions re: Asset retirement cost I haven't
# checked in yet and probably won't bc they are generally one-offs
try:
    arc = pudl_rmi.model_inputs.make_dep_arc(d)
    scaled_arc = pudl_rmi.model_inputs.scale_arc(arc, ppl)
except AttributeError:
    logger.info("this is not checked in rn")
    scaled_arc = None
        
try:
    ferc_deprish_eia = ferc_deprish_eia.merge(
        scaled_arc[['arc_by_plant']],
        right_index=True, left_index=True,
        how='left',
    )
except TypeError:
    logger.info("the ARC table isn't available rn bc the functions aren't checked in")

In [ ]:
# This (or a cleaned up version of it) should probably live in execute()

# choose data for the following columns from either the depreciation data
# or the ferc1 data.
cols_to_choose = [
    'plant_name_eia',
    'fraction_owned',
    'technology_description',
    'operational_status',
    'utility_id_pudl',
    'plant_id_pudl',
    'capacity_mw',
    'report_year'
]
ferc_deprish_eia = ferc_deprish_eia.drop(
    columns=[f"{x}_deprish" for x in cols_to_choose] + [f"{x}_ferc1" for x in cols_to_choose])
# add some ppl columns back into the output
ppl_cols_to_add = cols_to_choose + [
    'total_fuel_cost', 'installation_year', 'net_generation_mwh',
    'capacity_factor', 
]
ferc_deprish_eia = (
    ferc_deprish_eia# is there a better way to do this? record_id_eia is already in the previous index
    .reset_index().set_index(['record_id_eia'])
    .merge(
        ppl[ppl_cols_to_add],
        right_index=True, left_index=True,
        how='left'
    )
)

### Export Duke Jawn

In [ ]:
try:
    ferc_deprish_eia = pudl_rmi.model_inputs.append_non_plant_deprish_records(d, ferc_deprish_eia)
except AttributeError:
    logger.info("This function isn't checked in rn. Don't know where it will end up living yet")

duke_out = pudl_rmi.model_inputs.convert_to_model_format(
    ferc_deprish_eia.reset_index().assign(report_year=lambda x:  x.report_date.dt.year),
    pudl_out,
    util_ids_pudl=[90,97],
    years=[2018,2019,2020]
)
duke_out = duke_out.replace(
    {'Utility': {
        'Duke Energy Progress - (NC)': 'Duke Energy Progress',
        'Duke Energy Carolinas, LLC': 'Duke Energy Carolinas'},
    }
)
duke_out['Asset Status'] = duke_out['Asset Status'].str.capitalize()

duke_out.to_csv(pudl_rmi.OUTPUTS_DIR / f"current_owned_duke_{date.today().strftime('%Y-%m-%d')}.csv", index=False)